In [ ]:
pip install opendatasets


In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/khushikimmatka/ap-refined-optical-coherence-tomography-images")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jhanvi823
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/khushikimmatka/ap-refined-optical-coherence-tomography-images


In [ ]:
from google.colab import files
uploaded = files.upload()

# Once the file is uploaded, unzip it
import zipfile
zip_file_path = next(iter(uploaded))  # Get the uploaded file name
unzip_dir = '/content/glcm_features'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(f"Files extracted to {unzip_dir}")

Saving GLCM_FEATURES.zip to GLCM_FEATURES.zip
Files extracted to /content/glcm_features


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Concatenate, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

In [ ]:
# TPU/CPU/GPU strategy
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("Running on TPU!")
except:
    strategy = tf.distribute.get_strategy()
    print("Running on CPU/GPU!")

Running on CPU/GPU!


In [ ]:
# Paths
base_image_path = "/content/ap-refined-optical-coherence-tomography-images/OCT2017_ap"
base_glcm_path = "/content/glcm_features/GLCM_FEATURES"

# Build path maps (filename → .npy path)
def build_glcm_path_map(glcm_split_path):
    path_map = {}
    for class_dir in os.listdir(glcm_split_path):
        class_path = os.path.join(glcm_split_path, class_dir)
        for file in os.listdir(class_path):
            if file.endswith(".npy"):
                key = file.split(".")[0]
                path_map[key] = os.path.join(class_path, file)
    return path_map

glcm_train_paths = build_glcm_path_map(os.path.join(base_glcm_path, "Train"))
glcm_val_paths = build_glcm_path_map(os.path.join(base_glcm_path, "val"))

In [ ]:
# Determine feature dimension
glcm_sample_path = list(glcm_train_paths.values())[0]
glcm_feature_dim = np.load(glcm_sample_path).shape[0]

# Use ImageDataGenerators
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    os.path.join(base_image_path, "train"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    os.path.join(base_image_path, "val"),
    target_size=image_size,
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=False
)

# Fit Scaler only on subset (to reduce RAM)
sample_features = []
for path in list(glcm_train_paths.values())[:1000]:  # sample 1000 for fit
    sample_features.append(np.load(path))
scaler = StandardScaler().fit(np.array(sample_features))

# Generator that lazily loads and scales GLCM
def lazy_data_generator(image_generator, glcm_map, scaler):
    while True:
        images, labels = next(image_generator)
        file_paths = [image_generator.filepaths[idx] for idx in image_generator.index_array[:len(images)]]
        glcm_batch = []
        for path in file_paths:
            fname = os.path.splitext(os.path.basename(path))[0]
            feature_path = glcm_map.get(fname)
            if not feature_path:
                raise ValueError(f"Missing GLCM feature for {fname}")
            features = np.load(feature_path)
            features = scaler.transform([features])[0]
            glcm_batch.append(features.astype(np.float32))
        yield (images, np.array(glcm_batch)), labels.astype(np.int32)

# Wrap with tf.data
train_ds = tf.data.Dataset.from_generator(
    lambda: lazy_data_generator(train_gen, glcm_train_paths, scaler),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, glcm_feature_dim), dtype=tf.float32),
        ),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_generator(
    lambda: lazy_data_generator(val_gen, glcm_val_paths, scaler),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, glcm_feature_dim), dtype=tf.float32),
        ),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
).prefetch(tf.data.AUTOTUNE)

# Compute class weights
class_weights = compute_class_weight("balanced", classes=np.unique(train_gen.classes), y=train_gen.classes)
class_weight_dict = {i: w for i, w in enumerate(class_weights)}

# Model
with strategy.scope():
    resnet_base = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    resnet_output = AveragePooling2D(pool_size=(7, 7))(resnet_base.output)
    resnet_output = Flatten()(resnet_output)

    glcm_input = Input(shape=(glcm_feature_dim,))
    glcm_fc1 = Dense(128, activation="relu")(glcm_input)
    glcm_fc2 = Dense(64, activation="relu")(glcm_fc1)

    combined = Concatenate()([resnet_output, glcm_fc2])
    fc1 = Dense(128, activation="relu")(combined)
    dropout = Dropout(0.5)(fc1)
    output = Dense(len(np.unique(train_gen.classes)), activation="softmax")(dropout)

    model = Model(inputs=[resnet_base.input, glcm_input], outputs=output)

    # Simple fixed learning rate
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    model.summary()

Found 61300 images belonging to 4 classes.
Found 7661 images belonging to 4 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,868,100 (91.05 MB)

 Trainable params: 23,814,980 (90.85 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
callbacks = [
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint("best_model.keras", monitor='val_loss', save_best_only=True),
    ModelCheckpoint("checkpoint_epoch_{epoch:02d}.keras", save_freq='epoch'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-7),
    CSVLogger("training_log.csv")
]

# Train
steps_per_epoch = np.ceil(train_gen.samples / batch_size).astype(int)
val_steps = np.ceil(val_gen.samples / batch_size).astype(int)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    steps_per_epoch=steps_per_epoch,
    validation_steps=val_steps,
    epochs=40,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/40
1916/1916 ━━━━━━━━━━━━━━━━━━━━ 1295s 641ms/step - accuracy: 0.8436 - loss: 0.4641 - val_accuracy: 0.9433 - val_loss: 0.1761 - learning_rate: 1.0000e-04
Epoch 2/40
1916/1916 ━━━━━━━━━━━━━━━━━━━━ 1217s 635ms/step - accuracy: 0.9318 - loss: 0.2233 - val_accuracy: 0.9379 - val_loss: 0.1851 - learning_rate: 1.0000e-04
Epoch 3/40
1916/1916 ━━━━━━━━━━━━━━━━━━━━ 0s 548ms/step - accuracy: 0.9369 - loss: 0.1936
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
1916/1916 ━━━━━━━━━━━━━━━━━━━━ 1221s 637ms/step - accuracy: 0.9369 - loss: 0.1936 - val_accuracy: 0.9324 - val_loss: 0.1991 - learning_rate: 1.0000e-04
Epoch 4/40
1916/1916 ━━━━━━━━━━━━━━━━━━━━ 1196s 624ms/step - accuracy: 0.9519 - loss: 0.1516 - val_accuracy: 0.9521 - val_loss: 0.1441 - learning_rate: 5.0000e-05
Epoch 5/40
1916/1916 ━━━━━━━━━━━━━━━━━━━━ 1203s 628ms/step - accuracy: 0.9558 - loss: 0.1361 - val_accuracy: 0.9567 - val_loss: 0.1254 - learning_rate: 5.0000e-05
Epoch 6/40
1916/1916 ━━━━━━━━━